In [1]:
# Main pycaret
!pip install --ignore-installed pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 551.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 7.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.6/262.6 kB 6.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 16.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.6 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getti

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import tensorflow.keras.backend as K
print('Using TensorFlow version',tf.__version__)

# RESTRICT TENSORFLOW TO 8GB OF GPU RAM
# SO THAT WE HAVE 8GB RAM FOR RAPIDS
LIMIT = 8
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

Using TensorFlow version 2.6.4
We will restrict TensorFlow to max 8GB GPU RAM
then RAPIDS can use 8GB GPU RAM


## The train and test aggregation files are created from these two notebooks.
### [Notebook 1](https://www.kaggle.com/code/sravanneeli/train-file-pckl-creation-from-parquet-files)
### [Notebook 2](https://www.kaggle.com/code/sravanneeli/test-file-pckl-creation-from-parquet-files)

In [1]:
import cupy, cudf # GPU LIBRARIES
import numpy as np, pandas as pd # CPU LIBRARIES
import matplotlib.pyplot as plt, gc

In [2]:

np.random.seed(42)

In [3]:
train_df = pd.read_pickle('../input/amex-data-pckl-files/train_agg.pkl', compression='gzip')
train_labels = cudf.read_csv('../input/amex-default-prediction/train_labels.csv')

In [4]:
train_df.shape

(458913, 742)

In [ ]:
gc.collect()

In [ ]:
cat_cols = []
for col in train_df:
    if "last" in col:
        cat_cols.append(col)

In [ ]:
while True:
    try:
        from pycaret.classification import *
        break
    except:
        pass

In [ ]:
train_labels = train_labels.sort_values(by=['customer_ID']).drop('customer_ID', axis=1)
train_df = pd.concat([train_df, train_labels], axis=1)

In [ ]:
train_df.head()

In [ ]:
# COMPETITION METRIC FROM Konstantin Yakovlev
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
clf1 = setup(data = train_df.drop('customer_ID', axis=1),
             session_id=2286,
             fold=5,
             categorical_features=cat_cols,
             use_gpu=True,
             target = 'target',
             silent = True)

In [ ]:
add_metric("amex_customer_metric", name="Amex Metric", score_func=amex_metric_mod)

In [ ]:
del(train_df)
gc.collect()

In [ ]:
# model training
model = compare_models(include = ['lightgbm'])

In [ ]:
gc.collect()

In [ ]:
predict_model(model, raw_score=True)

In [ ]:
test_df = pd.read_pickle('../input/amex-data-pckl-files/test_agg.pkl', compression='gzip')

In [ ]:
y_pred = []
for i in range(0, test_df.shape[0], 10000):
    y_pred.extend(predict_model(model, data=test_df.iloc[i:i+10000, :],raw_score=True)['Score_1'].to_list())

In [ ]:
sub_df = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')

In [ ]:
sub_df['prediction'] = y_pred

In [ ]:
sub_df.to_csv('submission.csv', index=False)